# Data mean removal

In [14]:
import xarray as xr
import pandas as pd

# Assuming the file '20170901_UV_sample.nc' is available in the working directory
file_path = './wind/20170901_UV_sample.nc'

# Load the netCDF file
try:
    ds = xr.open_dataset(file_path)
except FileNotFoundError:
    error_message = "File not found. Please upload the file '20170901_UV_sample.nc' to proceed."
    ds = None
# Convert to DataFrame and then to CSV, if the file is found
if ds:
    # Convert to DataFrame
    df = ds.to_dataframe()

    # Save to CSV
    csv_file_path = './wind/20170901_UV_sample.csv'
    df.to_csv(csv_file_path)
    success_message = f"File converted to CSV and saved as '{csv_file_path}'."
else:
    success_message = ""

success_message

"File converted to CSV and saved as './wind/20170901_UV_sample.csv'."

In [16]:
ds

<xarray.Dataset>
Dimensions:                (longitude: 89, latitude: 109, time: 24)
Coordinates:
  * longitude              (longitude) float64 20.39 20.44 20.49 ... 24.91 24.96
  * latitude               (latitude) float64 -5.609 -5.557 ... -0.01073
  * time                   (time) datetime64[ns] 2017-09-01 ... 2017-09-01T23...
Data variables: (12/34)
    UGRD_10maboveground    (time, latitude, longitude) float32 -0.53 ... -0.65
    VGRD_10maboveground    (time, latitude, longitude) float32 -2.5 ... 0.49
    UGRD_20maboveground    (time, latitude, longitude) float32 -0.78 ... -0.7
    VGRD_20maboveground    (time, latitude, longitude) float32 -3.98 ... 0.54
    UGRD_30maboveground    (time, latitude, longitude) float32 -1.03 ... -0.74
    VGRD_30maboveground    (time, latitude, longitude) float32 -5.62 ... 0.57
    ...                     ...
    UGRD_800maboveground   (time, latitude, longitude) float32 1.02 1.12 ... 0.9
    VGRD_800maboveground   (time, latitude, longitude) float32 -4.43 ... 1.38
    UGRD_900maboveground   (time, latitude, longitude) float32 1.01 ... 1.29
    VGRD_900maboveground   (time, latitude, longitude) float32 -3.46 ... 1.41
    UGRD_1000maboveground  (time, latitude, longitude) float32 0.88 1.0 ... 1.66
    VGRD_1000maboveground  (time, latitude, longitude) float32 -2.4 ... 1.43
Attributes:
    CDI:                  Climate Data Interface version 1.8.1 (http://mpimet...
    history:              Tue Dec 19 15:17:51 2017: cdo sellonlatbox,20,25,-6...
    Conventions:          COARDS
    History:              created by wgrib2
    GRIB2_grid_template:  10
    CDO:                  Climate Data Operators version 1.8.1rc2 (http://mpi...

In [8]:
import pandas as pd
import numpy as np

file_path = './wind/20170901_UV_sample.csv'  # Replace with your CSV file path
df = pd.read_csv(file_path)
_data_array = df.to_numpy()
# _data_array = _data_array[::24, :][:, (0, 1, 3, 4)]
# (u^2 + v^2)^(0.5)
# _data_array[:, 4] = np.sqrt(_data_array[:, 2] ** 2 + _data_array[:, 3] ** 2)
# _data_array

## Data processing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

np.random.seed(42)

def process_and_split_data(meta_file, observation_file, train_ratio, test_ratio):
    """
    Process the datasets and split them into training and testing subsets separately for meta and observation data.

    :param meta_file: File path for the METAinfo dataset.
    :param observation_file: File path for the ppt.complete.Y001 dataset.
    :param train_ratio: Percentage of the data to be used for training (default 10%).
    :param test_ratio: Percentage of the data to be used for testing (default 1%).
    :return: None
    """
    # Read the datasets without headers
    meta_data = pd.read_csv(meta_file, header=None)
    observation_data = pd.read_csv(observation_file, header=None)
    
    # Assuming meta_data and observation_data are of the same length
    n_rows = len(meta_data)

    # Generate random indices
    all_indices = np.arange(n_rows)
    train_and_test_indices = np.random.choice(all_indices, size=int((train_ratio + test_ratio) * n_rows), replace=False)

    # Split indices into training and testing
    train_indices, test_indices = train_test_split(train_and_test_indices, test_size=test_ratio / (train_ratio + test_ratio), random_state=42)

    # Sample data using the same indices for both datasets
    meta_train = meta_data.iloc[train_indices]
    meta_test = meta_data.iloc[test_indices]
    observation_train = observation_data.iloc[train_indices]
    observation_test = observation_data.iloc[test_indices]

    # # Sample 11% of each dataset
    # meta_sample = meta_data.sample(frac=train_ratio + test_ratio)
    # observation_sample = observation_data.sample(frac=train_ratio + test_ratio)

    # # Split the sample data into 10% training and 1% testing for both datasets
    # meta_train, meta_test = train_test_split(meta_sample, test_size=test_ratio / (train_ratio + test_ratio), random_state = 42)
    # observation_train, observation_test = train_test_split(observation_sample, test_size=test_ratio / (train_ratio + test_ratio), random_state = 42)

    # Save the split data
    meta_train.to_csv(
        os.path.join(data_folder, 'meta_train_' + str(ratio)), 
        index=False, header=False)
    print(meta_train.shape)
    meta_test.to_csv(
        os.path.join(data_folder, 'meta_test_' + str(ratio)), 
        index=False, header=False)
    print(meta_test.shape)
    observation_train.to_csv(
        os.path.join(data_folder, 'observation_train_' + str(ratio)), 
        index=False, header=False)
    observation_test.to_csv(
        os.path.join(data_folder, 'observation_test_'+ str(ratio)), 
        index=False, header=False)

    print("Data processing complete. Files saved for both meta and observation datasets.")

data_folder = './soil_moist/'
ratio = 0.25
# Example usage
process_and_split_data(
    os.path.join(data_folder, 'METAinfo'), 
    os.path.join(data_folder,'ppt.complete.Y001'),
    ratio, ratio * 0.1
)
ratio = 0.125
# Example usage
process_and_split_data(
    os.path.join(data_folder, 'METAinfo'), 
    os.path.join(data_folder,'ppt.complete.Y001'),
    ratio, ratio * 0.1
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def plot_subplots_with_scatter(train_meta_file, train_obs_file, test_meta_file, test_obs_file):
    # Load the data
    train_meta = pd.read_csv(train_meta_file, header=None)
    train_obs = pd.read_csv(train_obs_file, header=None)
    test_meta = pd.read_csv(test_meta_file, header=None)
    test_obs = pd.read_csv(test_obs_file, header=None)
    
    # Convert data to numpy arrays if they're not already
    all_train_data = np.array(train_meta)
    all_train_obs = np.array(train_obs).flatten()  # Flatten in case the observations are in a 2D array
    all_test_data = np.array(test_meta)
    all_test_obs = np.array(test_obs).flatten()
    
    # Create subplots
    fig, axs = plt.subplots(1, 1, figsize=(14, 7))
    
    # Scatter plot for training data
    scatter_train = axs.scatter(all_train_data[:, 1] + 180, all_train_data[:, 0], c=all_train_obs, 
                            #  vmin=np.min(all_train_obs), vmax=np.max(all_train_obs), 
                             cmap="jet", s=3)
    fig.colorbar(scatter_train, ax=axs)
    # axs.set_title('Training Data')
    axs.set_xlabel('Longitude')
    axs.set_ylabel('Latitude')
    
    plt.tight_layout()
    plt.show()

# Example usage
# plot_subplots_with_scatter('meta_train.csv', 'observation_train.csv', 'meta_test.csv', 'observation_test.csv')

In [ ]:
plot_subplots_with_scatter(
    './soil_moist/meta_train_0.25', 
    './soil_moist/observation_train_0.25',
    './soil_moist/meta_test_0.25', 
    './soil_moist/observation_test_0.25'   
)

## Run the experiment with real dataset